In [109]:

def calculate_supplyinprofit_percentile(df, column='profit_relative', perc_window=1400, norm_window=1400, normalization='maxmin'):
    # Calculate percentile
    percentile = df[column].rolling(window=perc_window).apply(
        lambda x: pd.Series(x).rank(pct=True).iloc[-1]
    )
    
    if normalization == False:
        return percentile
    elif normalization == 'maxmin':
        max_val = percentile.rolling(window=norm_window).max()
        min_val = percentile.rolling(window=norm_window).min()
        normalized_percentile = 2 * (percentile - min_val) / (max_val - min_val) - 1
    elif normalization == 'z-score':
        mean = percentile.rolling(window=norm_window).mean()
        std = percentile.rolling(window=norm_window).std()
        z_score = (percentile - mean) / std
        # Scale z-score to -1 to +1 range
        normalized_percentile = z_score / z_score.abs().max()
    else:
        raise ValueError("Invalid normalization method. Choose 'False', 'maxmin', or 'z-score'.")
    
    return normalized_percentile

In [110]:
# Cryptocurrency Analysis Script v002

import requests
import pandas as pd
import numpy as np
import io
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Constants
API_KEY = '2lZRFGaqFiEYkzr7WUuT4EaoC1X'  # Replace with your actual API key
SINCE_DATE = int(datetime(2014, 1, 1).timestamp())  # Jan 1, 2015
UNTIL_DATE = int(datetime.now().timestamp())  # Current date

# URLs for fetching data
PRICE_URL = 'https://api.glassnode.com/v1/metrics/market/price_usd_close'
METRICS = [
    'https://api.glassnode.com/v1/metrics/indicators/investor_capitalization',
    'https://api.glassnode.com/v1/metrics/supply/current',
    'https://api.glassnode.com/v1/metrics/indicators/liveliness',
    'https://api.glassnode.com/v1/metrics/indicators/realized_profit_lth_account_based',
    'https://api.glassnode.com/v1/metrics/indicators/realized_loss_lth_account_based',
    'https://api.glassnode.com/v1/metrics/supply/profit_relative'
]

def fetch_glassnode_data(url, asset='BTC'):
    params = {
        'a': asset,
        's': SINCE_DATE,
        'u': UNTIL_DATE,
        'api_key': API_KEY,
        'f': 'CSV',
        'c': 'USD'
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        df = pd.read_csv(io.StringIO(response.text))
        metric_name = url.split('/')[-1]
        df.columns = ['t', metric_name]
        df['t'] = pd.to_datetime(df['t'], unit='s')
        df[metric_name] = pd.to_numeric(df[metric_name], errors='coerce')
        return df
    else:
        print(f"Failed to fetch data from {url}. Status code: {response.status_code}")
        return None

# Fetch and merge data
price_df = fetch_glassnode_data(PRICE_URL)
all_dfs = [price_df]
for metric_url in METRICS:
    metric_df = fetch_glassnode_data(metric_url)
    if metric_df is not None:
        all_dfs.append(metric_df)

merged_df = pd.concat(all_dfs, axis=1)
merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]
merged_df.set_index('t', inplace=True)

In [111]:

def calculate_aviv_percentile(df, columns=['investor_capitalization', 'current', 'liveliness', 'price_usd_close'], perc_window=1400, norm_window=1400, normalization='maxmin'):
    # Calculate AVIV
    aviv = df[columns[2]] * df[columns[1]] * df[columns[3]] / df[columns[0]]
    
    # Calculate percentile
    percentile = aviv.rolling(window=perc_window).apply(
        lambda x: pd.Series(x).rank(pct=True).iloc[-1]
    )
    
    if normalization == False:
        return percentile
    elif normalization == 'maxmin':
        max_val = percentile.rolling(window=norm_window).max()
        min_val = percentile.rolling(window=norm_window).min()
        normalized_percentile = 2 * (percentile - min_val) / (max_val - min_val) - 1
    elif normalization == 'z-score':
        mean = percentile.rolling(window=norm_window).mean()
        std = percentile.rolling(window=norm_window).std()
        z_score = (percentile - mean) / std
        # Scale z-score to -1 to +1 range
        normalized_percentile = z_score / z_score.abs().max()
    else:
        raise ValueError("Invalid normalization method. Choose 'False', 'maxmin', or 'z-score'.")
    
    return normalized_percentile

In [112]:

def calculate_lthrealized_plratio_logpercentile(df, columns=['realized_profit_lth_account_based', 'realized_loss_lth_account_based'], perc_window=1400, norm_window=1400, normalization='maxmin'):
    # Calculate LTH realized P/L ratio
    lth_realized_pl = df[columns[0]] / df[columns[1]]
    
    # Apply log transformation
    log_lth_realized_pl = np.log(lth_realized_pl)
    
    # Calculate percentile
    percentile = log_lth_realized_pl.rolling(window=perc_window).apply(
        lambda x: pd.Series(x).rank(pct=True).iloc[-1]
    )
    
    if normalization == False:
        return percentile
    elif normalization == 'maxmin':
        max_val = percentile.rolling(window=norm_window).max()
        min_val = percentile.rolling(window=norm_window).min()
        normalized_percentile = 2 * (percentile - min_val) / (max_val - min_val) - 1
    elif normalization == 'z-score':
        mean = percentile.rolling(window=norm_window).mean()
        std = percentile.rolling(window=norm_window).std()
        z_score = (percentile - mean) / std
        # Scale z-score to -1 to +1 range
        normalized_percentile = z_score / z_score.abs().max()
    else:
        raise ValueError("Invalid normalization method. Choose 'False', 'maxmin', or 'z-score'.")
    
    return normalized_percentile

In [113]:

# Apply the functions to the merged dataframe with maxmin normalization
merged_df['supplyinprofit_percentile_maxmin'] = calculate_supplyinprofit_percentile(merged_df, normalization='maxmin')
merged_df['aviv_percentile_maxmin'] = calculate_aviv_percentile(merged_df, normalization='maxmin')
merged_df['lthrealized_plratio_logpercentile_maxmin'] = calculate_lthrealized_plratio_logpercentile(merged_df, normalization='maxmin')

In [114]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def aggregate_indicators(df, columns, method='equal_weight'):
    """
    Aggregate financial behavior indicators from a DataFrame, excluding initial rows with NaN values.
    
    :param df: pandas DataFrame containing the indicators
    :param columns: list of column names to be aggregated
    :param method: 'equal_weight' or 'PCA'
    :return: pandas Series with the aggregated indicator
    """
    # Ensure all specified columns exist in the DataFrame
    if not all(col in df.columns for col in columns):
        raise ValueError("Some specified columns are not in the DataFrame")
    
    # Extract the relevant columns
    data = df[columns]
    
    if method == 'equal_weight':
        # For equal weight, we can use pandas mean which automatically skips NaN
        weight = 1 / len(columns)
        print(f"weights are all equal = {weight:.4f}")
        return data.mean(axis=1)
    
    elif method == 'PCA':
        # Remove rows with any NaN values
        data_clean = data.dropna()
        
        if len(data_clean) == 0:
            raise ValueError("No complete rows found after removing NaN values")
        
        # Standardize the data
        scaler = StandardScaler()
        data_scaled = scaler.fit_transform(data_clean)
        
        # Perform PCA
        pca = PCA(n_components=1)
        pca_result = pca.fit_transform(data_scaled)
        
        # Calculate weights from the first principal component
        weights = pca.components_[0] / np.sum(np.abs(pca.components_[0]))
        
        print("PCA weights:")
        for col, weight in zip(columns, weights):
            print(f"{col}: {weight:.4f}")
        
        # Calculate weighted sum for all rows, including those with NaN
        weighted_sum = data.mul(weights).sum(axis=1)
        
        # Normalize the weighted sum to be in the same range as input data
        min_val, max_val = data.min().min(), data.max().max()
        normalized_sum = (weighted_sum - weighted_sum.min()) / (weighted_sum.max() - weighted_sum.min())
        normalized_sum = normalized_sum * (max_val - min_val) + min_val
        
        return normalized_sum
    
    else:
        raise ValueError("Invalid method. Choose 'equal_weight' or 'PCA'")


In [115]:

merged_df['aggregate_indicator_equalweight']= aggregate_indicators(merged_df, columns=['lthrealized_plratio_logpercentile_maxmin','aviv_percentile_maxmin','supplyinprofit_percentile_maxmin'], method='equal_weight')
merged_df['aggregate_indicator_PCA']= aggregate_indicators(merged_df, columns=['lthrealized_plratio_logpercentile_maxmin','aviv_percentile_maxmin','supplyinprofit_percentile_maxmin'], method='PCA')

weights are all equal = 0.3333
PCA weights:
lthrealized_plratio_logpercentile_maxmin: 0.3316
aviv_percentile_maxmin: 0.3338
supplyinprofit_percentile_maxmin: 0.3346


In [116]:
def create_indicator_chart(merged_df, indicator_column, chart_title):
    # Filter data for the last year
    one_year_ago = datetime.now() - timedelta(days=365)
    merged_df_last_year = merged_df[merged_df.index > one_year_ago]

    # Define colors
    GREY_COLOR = 'rgba(128, 128, 128, 0.7)'  # Semi-transparent grey
    RED_COLOR = 'red'
    BLUE_COLOR = 'blue'

    # Create the visualization
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add price trace
    fig.add_trace(
        go.Scatter(x=merged_df_last_year.index, y=merged_df_last_year['price_usd_close'], name="Price USD", line=dict(color=GREY_COLOR, width=2), mode='lines'),
        secondary_y=False,
    )

    # Add indicator trace
    indicator = merged_df_last_year[indicator_column]
    
    # Add red color for positive values with transparent fill
    fig.add_trace(
        go.Scatter(
            x=merged_df_last_year.index,
            y=indicator.where(indicator > 0, 0),
            name=f"{indicator_column} (Positive)",
            line=dict(color=RED_COLOR, width=2),
            fill='tozeroy',
            fillcolor='rgba(255,0,0,0)',  # Transparent red
            mode='lines'
        ),
        secondary_y=True,
    )

    # Add blue color for negative values with transparent fill
    fig.add_trace(
        go.Scatter(
            x=merged_df_last_year.index,
            y=indicator.where(indicator < 0, 0),
            name=f"{indicator_column} (Negative)",
            line=dict(color=BLUE_COLOR, width=2),
            fill='tozeroy',
            fillcolor='rgba(0,0,255,0)',  # Transparent blue
            mode='lines'
        ),
        secondary_y=True,
    )

    # Add y=0 line on top (without adding to legend)
    fig.add_trace(
        go.Scatter(
            x=merged_df_last_year.index,
            y=[0] * len(merged_df_last_year),
            showlegend=False,
            line=dict(color=GREY_COLOR, width=2),
            hoverinfo='skip'
        ),
        secondary_y=True,
    )

    # Add vertical lines for every two months (Jan, Mar, May, Jul, Sep, Nov)
    for month in [1, 3, 5, 7, 9, 11]:
        for year in range(merged_df_last_year.index[0].year, merged_df_last_year.index[-1].year + 1):
            date = pd.Timestamp(year=year, month=month, day=1)
            if merged_df_last_year.index[0] <= date <= merged_df_last_year.index[-1]:
                fig.add_vline(x=date, line_dash="dash", line_color=GREY_COLOR, line_width=0.75, opacity=0.7)

    # Get the last value of the indicator
    last_value = indicator.iloc[-1]

    # Determine the color based on the last value
    indicator_color = RED_COLOR if last_value >= 0 else BLUE_COLOR

    # Add annotation for the last value
    fig.add_annotation(
        x=0.95,  # Place at the right edge of the chart
        y=last_value*0.88,  # Place at the vertical position of the last value
        xref="paper",
        yref="y2",  # Use the secondary y-axis for reference
        text=f"{last_value:.2f}",
        showarrow=False,
        font=dict(size=18, color=indicator_color),
        align="left",
        xanchor="left",
        yanchor="middle",
    )

    # Update layout
    fig.update_layout(
        title={
            'text': chart_title,
            'font': {'color': 'black', 'size': 18, 'weight': 'bold'}
        },
        showlegend=False,  # Remove legend
        hovermode="x unified",
        plot_bgcolor='white',
        paper_bgcolor='white',
        font={'color': 'black', 'size': 14},
        width=1000,  # Set width for 16:9 aspect ratio
        height=450,  # Set height for 16:9 aspect ratio
    )

    # Update axes
    fig.update_xaxes(
        showgrid=False, 
        tickfont={'color': 'black', 'size': 14},
        zeroline=False
    )
    fig.update_yaxes(
        showgrid=False, 
        secondary_y=False, 
        tickfont={'color': GREY_COLOR, 'size': 14},
        zeroline=False,
        showline=True,
        linecolor=GREY_COLOR,
        ticks='outside',
        tickcolor=GREY_COLOR,
        title_text='',
        title_font=dict(size=18)
    )
    fig.update_yaxes(
        showgrid=False, 
        secondary_y=True, 
        range=[-1, 1], 
        tickfont={'color': GREY_COLOR, 'size': 14},
        zeroline=False,
        showline=True,
        linecolor=GREY_COLOR,
        ticks='outside',
        side='right',
        tickcolor=GREY_COLOR,
        title_text='',
        title_font=dict(size=18)
    )

    return fig

In [117]:

# Assuming merged_df is your dataframe containing all the necessary data
# If you haven't created merged_df yet, you need to do that before this visualization part

# Create charts for each indicator
indicators = [
    ('supplyinprofit_percentile_maxmin', "Bitcoin: Supply in Profit"),
    ('aviv_percentile_maxmin', "Bitcoin: AVIV"),
    ('lthrealized_plratio_logpercentile_maxmin', "Bitcoin: LTH Realized P/L Ratio"),
    ('aggregate_indicator_equalweight', "Bitcoin: Aggregate Indicator Equal Weight"),
    ('aggregate_indicator_PCA', "Bitcoin: Aggregate Indicator PCA")
]

for indicator, title in indicators:
    fig = create_indicator_chart(merged_df, indicator, title)
    
    # Show the plot
    fig.show()
    
    # Optionally, save the plot as an HTML file
    pio.write_html(fig, file=f'bitcoin_analysis_{indicator}_last_year.html')

print("All charts have been displayed and saved as separate HTML files.")

All charts have been displayed and saved as separate HTML files.


In [118]:
merged_df.tail()

price_usd_close  investor_capitalization       current  \
t                                                                    
2024-10-07     62306.874634             5.662415e+11  1.233883e+12   
2024-10-08     62184.594129             5.715005e+11  1.229083e+12   
2024-10-09     60591.185790             5.716971e+11  1.198939e+12   
2024-10-10     60232.825863             5.713810e+11  1.189534e+12   
2024-10-11     62401.427159             5.730195e+11  1.234778e+12   

            liveliness  realized_profit_lth_account_based  \
t                                                           
2024-10-07    0.608285                       2.647942e+08   
2024-10-08    0.609136                       7.587768e+08   
2024-10-09    0.609147                       2.983150e+08   
2024-10-10    0.609096                       1.106383e+08   
2024-10-11    0.609316                       1.671568e+09   

            realized_loss_lth_account_based  profit_relative  \
t                                                              
2024-10-07                     9.172013e+06         0.815999   
2024-10-08                     1.272373e+08         0.803814   
2024-10-09                     5.888562e+07         0.763794   
2024-10-10                     7.980728e+07         0.755444   
2024-10-11                     1.289563e+08         0.824827   

            supplyinprofit_percentile_maxmin  aviv_percentile_maxmin  \
t                                                                      
2024-10-07                          0.244957                0.452361   
2024-10-08                          0.203170                0.435193   
2024-10-09                          0.036023                0.349356   
2024-10-10                          0.007205                0.332189   
2024-10-11                          0.283862                0.442060   

            lthrealized_plratio_logpercentile_maxmin  \
t                                                      
2024-10-07                                  0.137339   
2024-10-08                                 -0.138770   
2024-10-09                                 -0.168813   
2024-10-10                                 -0.432046   
2024-10-11                                 -0.004292   

            aggregate_indicator_equalweight  aggregate_indicator_PCA  
t                                                                     
2024-10-07                         0.278219                 0.295920  
2024-10-08                         0.166531                 0.181817  
2024-10-09                         0.072189                 0.084858  
2024-10-10                        -0.030884                -0.020417  
2024-10-11                         0.240543                 0.257584

In [119]:
merged_df.to_csv('bitcoin_analysis_data_lth.csv', index=True)

In [ ]:
import plotly.io as pio
import plotly.graph_objects as go

# Define the color codes
GREY_COLOR = 'rgba(128, 128, 128, 0.7)'  # Semi-transparent grey
GREEN_COLOR = 'green'  # Green color for positive changes
RED_COLOR = 'red'  # Red color for negative changes

def calculate_periodic_changes(df, column):
    # Calculate the percentage changes for weekly, monthly, and quarterly periods
    weekly_change = df[column].pct_change(7).iloc[-1] * 100  # 7 days for weekly change
    monthly_change = df[column].pct_change(30).iloc[-1] * 100  # 30 days for monthly change
    quarterly_change = df[column].pct_change(90).iloc[-1] * 100  # 90 days for quarterly change

    return weekly_change, monthly_change, quarterly_change

In [121]:
def create_bar_chart(merged_df, column):
    # Calculate percentage changes for the specified column
    weekly_change, monthly_change, quarterly_change = calculate_periodic_changes(merged_df, column)
    
    # Define the periods and changes in the new order: quarterly, monthly, weekly
    periods = ['Quarterly', 'Monthly', 'Weekly']
    changes = [quarterly_change, monthly_change, weekly_change]
    
    # Determine the color for each bar: GREEN_COLOR for positive, RED_COLOR for negative
    colors = [GREEN_COLOR if change > 0 else RED_COLOR for change in changes]

    # Create the bar chart with filled bars and increased roundness
    fig = go.Figure()
    fig.add_trace(
        go.Bar(
            x=periods,
            y=changes,
            marker=dict(
                color=colors,  # Use the colors list for fill
                line=dict(
                    color=colors,  # Use the same colors for outline
                    width=2  # Adjust the line width as needed
                ),
                cornerradius=20  # Increase the roundness of corners
            ),
            text=[f"{change:.2f}%" for change in changes],  # Add text labels with percentage values
            textposition='outside',
            textfont=dict(color=GREY_COLOR, size=16),  # Set the text color to grey and increase size
            width=0.5  # Adjust bar width to make them closer
        )
    )

    # Customize the layout
    fig.update_layout(
        title={
            'text': f"Rate of Change {column}",
            'font': {'color': 'black', 'size': 18, 'weight': 'bold'}
        },
        xaxis_title=None,  # Remove the x-axis title
        yaxis_title=None,  # Remove the y-axis title
        yaxis_ticksuffix="%",
        plot_bgcolor='white',
        paper_bgcolor='white',
        font={'color': 'black', 'size': 14},
        width=600,  # Keep width at 600 pixels
        height=600,  # Keep height at 600 pixels
        hovermode="x unified",
        bargap=0.2  # Reduce gap between bars
    )

    # Add a horizontal line at y = 0%
    fig.add_hline(y=0, line_dash="solid", line_color=GREY_COLOR, line_width=1)

    # Set y-axis tickers to every 40%
    fig.update_yaxes(
        showgrid=False, 
        tickfont={'color': GREY_COLOR, 'size': 14},  # Make y-axis labels grey
        zeroline=False,
        linecolor=GREY_COLOR,  # Grey color for y-axis lines
        showline=True,
        ticks='outside',
        tickcolor=GREY_COLOR,
        dtick=40  # Set dtick to 40 to display tickers every 40%
    )

    # Update x-axis (periods)
    fig.update_xaxes(
        showgrid=False, 
        tickfont={'color': GREY_COLOR, 'size': 14},  # Make x-axis labels grey
        zeroline=False,
        linecolor=None,  # Remove x-axis line
        showline=False,  # Ensure no line on x-axis
        ticks='outside',
        tickcolor=GREY_COLOR
    )

    # Show the chart
    fig.show()

    # Save the chart as an HTML file
    pio.write_html(fig, file=f'{column}_periodic_changes_chart.html')

In [122]:
# Call the function for aggregate indicator columns and save charts
create_bar_chart(merged_df, 'aggregate_indicator_equalweight')
create_bar_chart(merged_df, 'aggregate_indicator_PCA')